In [ ]:
import os
import torch
from pytube import YouTube
from moviepy.editor import VideoFileClip
from transformers import pipeline,T5Tokenizer,T5ForConditionalGeneration
import torchaudio
from torchaudio.transforms import Resample

In [ ]:
pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 997.1 kB/s eta 0:00:00


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00


In [ ]:
def download_youtube_video(link, output_file):
    yt = YouTube(link)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path='', filename=output_file)

In [ ]:
def convert_to_wav(input_video, output_audio):
    video_clip = VideoFileClip(input_video)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio, codec="pcm_s16le")
    audio_clip.close()
    video_clip.close()

In [ ]:
def process_audio_for_transcription(audio_file):
    waveform, sample_rate = torchaudio.load(audio_file)

    # Convert to single-channel (mono) if the audio is multi-channel
    if waveform.size(0) > 1:
        # Take the average of the channels along the channel dimension (dim=0)
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Resample the audio to the model's expected sample rate
    resampler = Resample(orig_freq=sample_rate, new_freq=16000)
    resampled_waveform = resampler(waveform)

    # Resample the waveform to the model's expected sample rate
    resampled_audio = resampled_waveform.squeeze(0).numpy()

    return resampled_audio

In [ ]:
if __name__ == "__main__":
    youtube_link = input("Enter YouTube video link: ")
    video_file = "video.mp4"
    audio_file = "audio.wav"
    transcript = ""

    try:
        # Download the YouTube video
        download_youtube_video(youtube_link, video_file)
        print("Video downloaded successfully.")

        # Convert video to WAV format
        convert_to_wav(video_file, audio_file)
        print("Audio converted to WAV format successfully.")

        # Process audio for transcription
        audio_data = process_audio_for_transcription(audio_file)

        # Get audio length in seconds
        audio_info = torchaudio.info(audio_file)
        audio_length = audio_info.num_frames / audio_info.sample_rate

        # Transcription
        transcribe = pipeline(
            task="automatic-speech-recognition",
            model="vasista22/whisper-hindi-small",
            chunk_length_s=audio_length,  # Set chunk_length_s equal to audio length
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="hi", task="transcribe")

        print("Transcription (Hindi):", transcribe(audio_data)["text"])

    except Exception as e:
        print(f"An error occurred: {e}")


Enter YouTube video link: https://www.youtube.com/watch?v=n0NWsLoMWcs
Video downloaded successfully.
MoviePy - Writing audio in audio.wav


MoviePy - Done.
Audio converted to WAV format successfully.
Transcription (Hindi): अगर � हाथ की लखी रोग नहीं बदलते दूर नहीं है तब आपके पास सब कुछ होगा लेकिन फिर भी अपना कुछ भी होने का डर नहीं


In [ ]:


import os
import torch
from pytube import YouTube
from moviepy.editor import VideoFileClip
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
import torchaudio
from torchaudio.transforms import Resample

In [ ]:
def download_youtube_video(link, output_file):
    yt = YouTube(link)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path='', filename=output_file)

In [ ]:
def convert_to_wav(input_video, output_audio):
    video_clip = VideoFileClip(input_video)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio, codec="pcm_s16le")
    audio_clip.close()
    video_clip.close()

In [ ]:
def process_audio_for_transcription(audio_file):
    waveform, sample_rate = torchaudio.load(audio_file)

    # Convert to single-channel (mono) if the audio is multi-channel
    if waveform.size(0) > 1:
        # Take the average of the channels along the channel dimension (dim=0)
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Resample the audio to the model's expected sample rate
    resampler = Resample(orig_freq=sample_rate, new_freq=16000)
    resampled_waveform = resampler(waveform)

    # Resample the waveform to the model's expected sample rate
    resampled_audio = resampled_waveform.squeeze(0).numpy()

    return resampled_audio

In [ ]:
def summarize_hindi_transcription(transcription):
    # Load the T5 tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")

    # Prepare the input
    input_text = "summarize: " + transcription
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs, num_beams=4, min_length=30, max_length=150, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [ ]:
if __name__ == "__main__":
    youtube_link = input("Enter YouTube video link: ")
    video_file = "video.mp4"
    audio_file = "audio.wav"
    transcript = ""

    try:
        # Download the YouTube video
        download_youtube_video(youtube_link, video_file)
        print("Video downloaded successfully.")

        # Convert video to WAV format
        convert_to_wav(video_file, audio_file)
        print("Audio converted to WAV format successfully.")

        # Process audio for transcription
        audio_data = process_audio_for_transcription(audio_file)

        # Transcription
        transcribe = pipeline(
            task="automatic-speech-recognition",
            model="vasista22/whisper-hindi-small",
            chunk_length_s=audio_length,  # Set chunk_length_s equal to audio length
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="hi", task="transcribe")

        hindi_transcription = transcribe(audio_data)["text"]
        print("Transcription (Hindi):", hindi_transcription)

        # Summarization
        summarized_text = summarize_hindi_transcription(hindi_transcription)

        print("Summarized Text:")
        print(summarized_text)

    except Exception as e:
        print(f"An error occurred: {e}")

Enter YouTube video link: https://www.youtube.com/watch?v=n0NWsLoMWcs
Video downloaded successfully.
MoviePy - Writing audio in audio.wav


MoviePy - Done.
Audio converted to WAV format successfully.
Transcription (Hindi): अगर � हाथ की लखी रोग नहीं बदलते दूर नहीं है तब आपके पास सब कुछ होगा लेकिन फिर भी अपना कुछ भी होने का डर नहीं
An error occurred: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.



In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00


In [ ]:
if __name__ == "__main__":
    youtube_link = input("Enter YouTube video link: ")
    video_file = "video.mp4"
    audio_file = "audio.wav"
    transcript = ""

    try:
        # Download the YouTube video
        download_youtube_video(youtube_link, video_file)
        print("Video downloaded successfully.")

        # Convert video to WAV format
        convert_to_wav(video_file, audio_file)
        print("Audio converted to WAV format successfully.")

        # Process audio for transcription
        audio_data = process_audio_for_transcription(audio_file)

        # Transcription
        transcribe = pipeline(
            task="automatic-speech-recognition",
            model="vasista22/whisper-hindi-small",
            chunk_length_s=audio_length,  # Set chunk_length_s equal to audio length
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="hi", task="transcribe")
        # (Code remains the same)

        # Transcription
        # (Code remains the same)

        print("Transcription (Hindi):", transcribe(audio_data)["text"])

        # Summarization
        hindi_transcription = transcribe(audio_data)["text"]
        summarized_text = summarize_hindi_transcription(hindi_transcription)

        print("Summarized Text:")
        print(summarized_text)

    except Exception as e:
        print(f"An error occurred: {e}")

Video downloaded successfully.
MoviePy - Writing audio in audio.wav


MoviePy - Done.
Audio converted to WAV format successfully.


  warnings.warn(



Transcription (Hindi): मैं कहां से खराब आए और हम सबने कहा कि आपकी प्रॉब्लम नहीं है हम सबकी प्रॉब्छा होता है परचे बनवाए थे अपने फ्रेंड में रिलेटेज में यहाँ पे वहाँ पे देते थे वो इतन� नहीं � यहाँ से तीसरी सीट पर बैठा हुई लाख � हूँदो दो लाख के चेक ऐैं पागल हो लेकर के जाता था वो मेरा और ज्यादा मजाक उठाते अब ये जो मैं फेल हुआ वो मेरी अच्छेमेरे को मेरे रिलेटिव ने फैमिली मेंऔरखादू तो आपको हँसते पेड़ दर्ज हो जाएपनी सक्सेस की वजह सीखनी है मुझे फोटोग्राफ आती है ना तो लाइफ में कभी न कभी ऐसी स्टेज आएगी आप लोग लाइफ में आप उठा लोगे एक्शन ले लोगे और जो आप चाहते हो वो जाकर रियलिटी से टकराएगा जो बहुत अलग ह�इंट था कि मेरे फेलियर मेरी सक्सेस कैसे एक छोटी सी ऐड निकाली न्यूजझेअपनी पहली आइड द्टी बहुद प्रिंटर को थोड़ा सा एडवां के साथ निकल गया
An error occurred: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environm

In [24]:
import os
import torch
from pytube import YouTube
from moviepy.editor import VideoFileClip
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
import torchaudio
from torchaudio.transforms import Resample
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

In [25]:
def download_youtube_video(link, output_file):
    yt = YouTube(link)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path='', filename=output_file)

In [26]:
def convert_to_wav(input_video, output_audio):
    video_clip = VideoFileClip(input_video)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio, codec="pcm_s16le")
    audio_clip.close()
    video_clip.close()

In [27]:
def process_audio_for_transcription(audio_file):
    waveform, sample_rate = torchaudio.load(audio_file)

    # Convert to single-channel (mono) if the audio is multi-channel
    if waveform.size(0) > 1:
        # Take the average of the channels along the channel dimension (dim=0)
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Resample the audio to the model's expected sample rate
    resampler = Resample(orig_freq=sample_rate, new_freq=16000)
    resampled_waveform = resampler(waveform)

    # Resample the waveform to the model's expected sample rate
    resampled_audio = resampled_waveform.squeeze(0).numpy()

    return resampled_audio

In [29]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

def summarize_hindi_transcription(transcription):
    # Load the T5 tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")

    # Prepare the input
    input_text = "summarize: " + transcription
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs, num_beams=4, min_length=30, max_length=150, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


In [31]:
if __name__ == "__main__":
    youtube_link = input("Enter YouTube video link: ")
    video_file = "video.mp4"
    audio_file = "audio.wav"
    transcript = ""

    try:
      # Download the YouTube video
        download_youtube_video(youtube_link, video_file)
        print("Video downloaded successfully.")

        # Convert video to WAV format
        convert_to_wav(video_file, audio_file)
        print("Audio converted to WAV format successfully.")

        # Process audio for transcription
        audio_data = process_audio_for_transcription(audio_file)

        # Transcription
        transcribe = pipeline(
            task="automatic-speech-recognition",
            model="vasista22/whisper-hindi-small",
            chunk_length_s=audio_length,  # Set chunk_length_s equal to audio length
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="hi", task="transcribe")
        # (Code remains the same)

        # Transcription
        # (Code remains the same)
        # (Code remains the same)

        # Transcription
        # (Code remains the same)

        print("Transcription (Hindi):", transcribe(audio_data)["text"])

        # Summarization
        hindi_transcription = transcribe(audio_data)["text"]
        summarized_text = summarize_hindi_transcription(hindi_transcription)

        print("Summarized Text:")
        print(summarized_text)

    except Exception as e:
        print(f"An error occurred: {e}")

Enter YouTube video link: https://www.youtube.com/watch?v=MgWrHsRaMv0
Video downloaded successfully.
MoviePy - Writing audio in audio.wav


MoviePy - Done.
Audio converted to WAV format successfully.
Transcription (Hindi): मैं कहां से खराब आए और हम सबने कहा कि आपकी प्रॉब्लम नहीं है हम सबकी प्रॉब्छा होता है परचे बनवाए थे अपने फ्रेंड में रिलेटेज में यहाँ पे वहाँ पे देते थे वो इतन� नहीं � यहाँ से तीसरी सीट पर बैठा हुई लाख � हूँदो दो लाख के चेक ऐैं पागल हो लेकर के जाता था वो मेरा और ज्यादा मजाक उठाते अब ये जो मैं फेल हुआ वो मेरी अच्छेमेरे को मेरे रिलेटिव ने फैमिली मेंऔरखादू तो आपको हँसते पेड़ दर्ज हो जाएपनी सक्सेस की वजह सीखनी है मुझे फोटोग्राफ आती है ना तो लाइफ में कभी न कभी ऐसी स्टेज आएगी आप लोग लाइफ में आप उठा लोगे एक्शन ले लोगे और जो आप चाहते हो वो जाकर रियलिटी से टकराएगा जो बहुत अलग ह�इंट था कि मेरे फेलियर मेरी सक्सेस कैसे एक छोटी सी ऐड निकाली न्यूजझेअपनी पहली आइड द्टी बहुद प्रिंटर को थोड़ा सा एडवां के साथ निकल गया
An error occurred: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepie